In [131]:
! pip install tensorflow numpy matplotlib pandas scikit-learn opencv-python ipywidgets

import numpy as np
import matplotlib.pyplot as plt
import os
from glob import glob
import tensorflow as tf
import keras
from keras import Input, Model
from keras.layers import Conv2D, Concatenate, Flatten, Dense, MaxPooling2D
import keras.optimizers
import sklearn
import sklearn.metrics
from PIL import Image
import time
from IPython.display import clear_output, display
import math
import ipywidgets as widgets
from IPython.display import display, clear_output
import cv2




In [132]:
def load_images_from_folder(folder, image_size=(500, 500), numImgs = (0, 100)):
    paths = sorted(glob(os.path.join(folder, '*.png')) + glob(os.path.join(folder, '*.jpg')))
    paths = sorted(paths, key=lambda x:int(os.path.basename(x).split('.')[0]))

    paths = paths[numImgs[0]:numImgs[1]]

    images = []
    for path in paths:
        img = keras.preprocessing.image.load_img(path, target_size=image_size)
        img = keras.preprocessing.image.img_to_array(img).astype(np.float32)
        img = img / 255.0  # Normalize to [0,1]
        images.append(img)
    return np.array(images, dtype=np.float32)


# [[distance, pitch, yaw, vehicle_id_string],...]
def load_transforms(folder, numImgs = (0, 100)):
    paths = sorted(glob(os.path.join(folder, '*.npy')))
    paths = sorted(paths, key=lambda x: int(os.path.basename(x).split('.')[0]))

    paths = paths[numImgs[0]:numImgs[1]]

    transforms = []
    for path in paths:
        data = np.load(path) 
        transforms.append(data)
    
    return np.array(transforms)


dataset_folder = 'sample_dataset'
imgsToLoad = (101, 209)
sample_references = load_images_from_folder(f"{dataset_folder}/reference", numImgs = imgsToLoad)
sample_masks = load_images_from_folder(f"{dataset_folder}/masks", numImgs = imgsToLoad)
sample_overlays = load_images_from_folder(f"{dataset_folder}/overlays", numImgs = imgsToLoad)
sample_transforms = load_transforms(f"{dataset_folder}/transforms", numImgs = imgsToLoad)



In [133]:
# all the stuff to calculate the transformation matrices

def rotation_x(theta):
    """Roll: Rotation about the X-axis"""
    c, s = np.cos(theta), np.sin(theta)
    return np.array([[ c, 0, s],
                     [ 0, 1, 0],
                     [-s, 0, c]])



def rotation_y(theta):
    """Pitch: Rotation about the Y-axis"""
    c, s = np.cos(theta), np.sin(theta)
    return np.array([[1, 0,  0],
                     [0, c, -s],
                     [0, s,  c]])


def rotation_z(theta):
    """Yaw: Rotation about the Z-axis"""
    c, s = np.cos(theta), np.sin(theta)
    return np.array([[c, -s, 0],
                     [s,  c, 0],
                     [0,  0, 1]])


def calcTransforms(pitch, yaw):
    yawdiff = ((yaw + 45) % 90) - 45 #the difference of yaw from the nearest 90 degree multiple
    if pitch > 35: # top face is predominant
        pitch = -pitch
        yaw = -yawdiff
        roll = 0

    elif pitch < 15: # lowest orbit angle
        pitch = pitch
        roll = -yawdiff
        yaw = 0
    
    else: # mid orbit angle
        #if on front face
        if 145 < yaw < 215:
            yaw = -yawdiff
            pitch = -pitch
            roll = 0

        else:
            roll = -yawdiff
            yaw = 0

    return pitch, yaw, roll



def calc3dRotMat(pitch, yaw):
    pitch, yaw, roll = calcTransforms(pitch, yaw)


    pitch_rad = np.radians(pitch)
    roll_rad = np.radians(roll)
    yaw_rad = np.radians(yaw)

    res_mat = np.eye(3)

    #pitch by pitch
    pitch_mat = rotation_y(pitch_rad)

    #roll by roll
    roll_mat = rotation_x(roll_rad)

    # yaw by yawdiff?
    yaw_mat = rotation_z(yaw_rad)

    res_mat =  pitch_mat @ roll_mat @ yaw_mat @ res_mat

    return res_mat



       

In [134]:
def transform_projection(image, pitch, yaw, distance, shift=(0,0)):

    h, w = image.shape[:2]
    shift_x, shift_y = shift

    scale = 15 - distance

    # ---- Projection parameters ----
    f = 500

    # Output size and center (origin in projection space)
    out_w, out_h = 500, 500
    out_cx, out_cy = out_w // 2, out_h // 2

    # ---- Define quad centered at origin ----
    corners_3d = np.array([
        [-w/2, -h/2, 0],
        [ w/2, -h/2, 0],
        [ w/2,  h/2, 0],
        [-w/2,  h/2, 0]
    ], dtype=np.float32)

    # ---- Rotate (rotation matrix keeps origin fixed as you noted) ----
    R = calc3dRotMat(pitch, yaw)
    rotated = (R @ corners_3d.T).T

    # ---- Scale ----
    rotated *= scale

    # ---- Perspective projection with origin at output center ----
    projected = rotated.copy()
    projected[:, 0] = f * projected[:, 0] / (f + projected[:, 2]) + out_cx + shift_x
    projected[:, 1] = f * projected[:, 1] / (f + projected[:, 2]) + out_cy + shift_y

    # ---- Warp ----
    src_pts = np.array([
        [0, 0],
        [w, 0],
        [w, h],
        [0, h]
    ], dtype=np.float32)

    dst_pts = projected[:, :2].astype(np.float32)

    M = cv2.getPerspectiveTransform(src_pts, dst_pts)

    output = cv2.warpPerspective(
        image, M, (out_w, out_h),
        flags=cv2.INTER_NEAREST
    )

    return output.astype(np.float32) / 255.0


In [135]:
textureStyle = 'noise'
textureResolution = 64
texture = np.random.randint(0, 256, (textureResolution, textureResolution, 3), dtype=np.uint8)


display_widget = widgets.Image(format='jpeg', width=1000)
info_label = widgets.HTML(value="<b>Initializing...</b>")

sample_slider = widgets.IntSlider(value=16, min=0, max=len(sample_references)-1, description='Sample:', continuous_update=False)

def to_uint8(img):
        return (np.clip(img, 0, 1) * 255).astype(np.uint8)


def fast_render(sampleNo):
    # Load data
    ref_img   = sample_references[sampleNo].astype(np.float32)
    mask_img  = sample_masks[sampleNo]
    overlay   = sample_overlays[sampleNo]
    transforms = sample_transforms[sampleNo]
    #[[distance, pitch, yaw, vehicle_id_string],...]

    # Transform texture
    transformed_tex = transform_projection(texture, int(transforms[1]), int(transforms[2]), int(transforms[0]))
    

    # Build mask (road but not car)
    road_mask = np.any(mask_img > 0.01, axis=-1, keepdims=True)
    car_mask  = np.any(overlay  > 0.01, axis=-1, keepdims=True)
    apply_tex = road_mask & ~car_mask

    # Composite
    final_comp = np.where(apply_tex, transformed_tex, ref_img)

    # Preview strip
    combined = np.hstack((
        to_uint8(ref_img),
        to_uint8(transformed_tex),
        to_uint8(final_comp)
    ))

    # Encode and display
    _, encoded = cv2.imencode(
        '.jpg',
        cv2.cvtColor(combined, cv2.COLOR_RGB2BGR),
        [int(cv2.IMWRITE_JPEG_QUALITY), 80]
    )
    display_widget.value = encoded.tobytes()

    # Info panel
    info_label.value = f"""
    <div style="font-family: monospace; font-size: 14px; line-height: 1.5;">
        <b>Original Data:</b> {transforms} (Dist, Pitch, Yaw)
    </div>
    """



# Hook up UI
out = widgets.interactive_output(fast_render, {'sampleNo': sample_slider})
ui_controls = widgets.VBox([sample_slider, info_label])

display(ui_controls, display_widget, out)



Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xdb\x00C\x00\x06\x04\x0…

Output()